read info.json

In [5]:
import json
import os
f = 'B075X65R3X/info.json'
with open(f, 'r') as file:
    data = json.load(file)

# Access the data
data

{'backend': 'CYCLES',
 'light_mode': 'uniform',
 'fast_mode': False,
 'format_version': 6,
 'channels': ['R', 'G', 'B', 'A', 'D'],
 'scale': 0.5,
 'images': [{'intrinsic': [[711.1110599640117, 0.0, 256.0],
    [0.0, 711.1110599640117, 256.0],
    [0.0, 0.0, 1.0]],
   'pose': [[-0.8608624339103699,
     -0.3795045018196106,
     0.3389574885368347,
     0.6779149174690247],
    [0.5088377594947815,
     -0.6420537829399109,
     0.5734554529190063,
     1.1469107866287231],
    [1.093386803319163e-08,
     0.666140615940094,
     0.7458261847496033,
     1.491652488708496],
    [0.0, 0.0, 0.0, 1.0]],
   'rgb': '00000_rgb.png',
   'depth': '00000_depth.png',
   'alpha': '00000_alpha.png',
   'max_depth': 5.0,
   'HW': [512, 512]},
  {'intrinsic': [[711.1110599640117, 0.0, 256.0],
    [0.0, 711.1110599640117, 256.0],
    [0.0, 0.0, 1.0]],
   'pose': [[-0.7480717301368713,
     0.21397116780281067,
     0.6281759738922119,
     1.2563519477844238],
    [0.6636179089546204,
     0.241201877

In [61]:
import numpy as np
import collections
import struct
import math

CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"])
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])
CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12)
}
CAMERA_MODEL_IDS = dict([(camera_model.model_id, camera_model)
                         for camera_model in CAMERA_MODELS])
CAMERA_MODEL_NAMES = dict([(camera_model.model_name, camera_model)
                           for camera_model in CAMERA_MODELS])

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

def fov2focal(fov, pixels):
    return pixels / (2 * math.tan(fov / 2))

def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def read_intrinsics_text(path):
    """
    Taken from https://github.com/colmap/colmap/blob/dev/scripts/python/read_write_model.py
    """
    cameras = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                camera_id = int(elems[0])
                model = elems[1]
                # assert model == "PINHOLE", "While the loader support other types, the rest of the code assumes PINHOLE"
                width = int(elems[2])
                height = int(elems[3])
                params = np.array(tuple(map(float, elems[4:])))
                cameras[camera_id] = Camera(id=camera_id, model=model,
                                            width=width, height=height,
                                            params=params)
    return cameras

def read_extrinsics_text(path):
    """
    Taken from https://github.com/colmap/colmap/blob/dev/scripts/python/read_write_model.py
    """
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack([tuple(map(float, elems[0::3])),
                                       tuple(map(float, elems[1::3]))])
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)
    return images
    
def intr_from_pinhole_camera(camera):
    if camera.model=='SIMPLE_PINHOLE':
        # https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h#L250-L256
        f, cx, cy =  camera.params
        fx = fy = f
    elif camera.model=='PINHOLE':
        # https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h#L265-L271
        fx, fy, cx, cy =  camera.params
    return np.array([
                [fx, 0, camera.width/2],
                [0, fy, camera.height/2],
                [0, 0, 1]
            ]).tolist()

def hw_from_pinhole_camera(camera):
    return [camera.height, camera.width]

def extr_from_image(image):
    R = image.qvec2rotmat()
    t = np.array(image.tvec)
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0 
    return Rt.tolist()

In [63]:
path = 'aparecida5'
cameras_extrinsic_file = os.path.join(path, "sparse/0", "images.txt")
cam_extrinsics = read_extrinsics_text(cameras_extrinsic_file)

cameras_intrinsic_file = os.path.join(path, "sparse/0", "cameras.txt")
cam_intrinsics = read_intrinsics_text(cameras_intrinsic_file)

images_list = []
for k, v in cam_extrinsics.items():
    pose = extr_from_image(v)
    image = {
        'intrinsic':intr_from_pinhole_camera(cam_intrinsics[v.camera_id]),
        'pose': pose,
        'HW': hw_from_pinhole_camera(cam_intrinsics[v.camera_id]) }
    images_list.append( image )
# cam_extrinsics
cam_extrinsics

{1: Image(id=1, qvec=array([ 0.99887792, -0.0129593 ,  0.03826974, -0.02470593]), tvec=array([-1.61229348, -0.20723563,  2.18756859]), camera_id=1, name='20250105_164818.jpg', xys=array([[3794.08129883,    4.91670752],
        [3794.08129883,    4.91670752],
        [  65.73661041,   19.34813881],
        ...,
        [1472.27661133, 1167.29333496],
        [3781.79052734, 1140.13598633],
        [2690.58740234, 1534.77185059]]), point3D_ids=array([  -1,   -1,   -1, ...,   -1,   -1, 2490])),
 2: Image(id=2, qvec=array([ 0.9925519 ,  0.08399517, -0.03659143,  0.08029081]), tvec=array([-1.80931678, -0.32511188,  3.51023525]), camera_id=1, name='20250105_164835.jpg', xys=array([[1.05630542e+03, 3.04900765e+00],
        [1.06357837e+03, 3.06640649e+00],
        [1.56434509e+03, 5.13650608e+00],
        ...,
        [1.55971777e+03, 5.49968140e+02],
        [3.39619019e+03, 1.11845935e+03],
        [3.39619019e+03, 1.11845935e+03]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1])),
 3: Im

In [ ]:
"""
Script to parse the generated files into our desirable format
"""
import argparse
import json
import math
import os
import random
import sys
import glob
from PIL import Image
import numpy as np
import pdb

def main():

    parser = argparse.ArgumentParser()
    parser.add_argument("--input_path", required=True, type=str)
    parser.add_argument("--output_path", required=True, type=str)
    args = parser.parse_args()
    
    input_path = args.input_path
    output_path = args.output_path
    os.makedirs(output_path, exist_ok=True)
    info = json.load(open(os.path.join(input_path, 'info.json')))
    
    object = dict(images=[])
    num_images = len(glob.glob(f'{input_path}/*[0-9].json'))
    for i in range(num_images):
        camera_metadata = json.load(open(os.path.join(input_path, f"{i:05}.json")))
        channels = [np.array(Image.open(os.path.join(input_path, f"{i:05}_{c}.png"))) / (2**16-1) for c in ['r', 'g', 'b', 'a', 'depth']]
        
        # mapping to [0,255]
        rgb = np.uint8(np.stack(channels[:3], axis=-1) * 255)
        mask = np.uint8(channels[3] * 255)
        depth = np.uint8(channels[-1] * 255)

        # merge image
        Image.fromarray(rgb).convert('RGB').save(os.path.join(output_path, f'{i:05}_rgb.png'))
        Image.fromarray(depth).save(os.path.join(output_path, f'{i:05}_depth.png'))
        Image.fromarray(mask).save(os.path.join(output_path, f'{i:05}_alpha.png'))

        # intrinsic
        x_fov, y_fov = camera_metadata['x_fov'], camera_metadata['y_fov']
        width, height = rgb.shape[:2]
        fx = 1 / np.tan(x_fov / 2) * (width / 2)
        fy = 1 / np.tan(y_fov / 2) * (height / 2)
        intrinsic = np.array([
            [fx, 0, width/2],
            [0, fy, height/2],
            [0, 0, 1]
        ])

        """
            OpenGL (X right; Y up; Z inward) 
        """

        # write extrinsic from opencv camera
        origin = np.array(camera_metadata['origin']).reshape(-1,1) # origin
        right = np.array(camera_metadata['x']).reshape(-1,1)  # right vector
        down = np.array(camera_metadata['y']).reshape(-1,1)   # down vector
        lookat = np.array(camera_metadata['z']).reshape(-1,1) # forward vector

        pose = np.block([
                [right, -down,  -lookat,  origin],
                [0., 0., 0., 1.]]
            )

        image = dict(
            intrinsic= intrinsic.tolist(),
            pose = pose.tolist(),
            rgb=os.path.join(f'{i:05}_rgb.png'),
            depth=os.path.join(f'{i:05}_depth.png'),
            alpha=os.path.join(f'{i:05}_alpha.png'),
            max_depth = camera_metadata['max_depth'],
            HW = [height, width],
        )

        object['images'].append(image)
        object['bbox'] = camera_metadata['bbox'] 

    with open(os.path.join(output_path, 'info.json'), "w") as f:
        object = {**info, **object}
        json.dump(object, f)

main()

ndle pkgconf:x64-windows: 21 s
pkgconf:x64-windows package ABI: d6f0eec77745829d2e4a558b85d0f7aad23a24db219d4ea95ea1ddfdc9cbe217
Completed submission of pcre2[core,jit,platform-default-features]:x64-windows@10.45 to 1 binary cache(s) in 727 ms
Installing 138/142 vcpkg-pkgconfig-get-modules:x64-windows@2024-04-03...
Building vcpkg-pkgconfig-get-modules:x64-windows@2024-04-03...
-- Installing: D:/Mestrado/Dissertation/labs/vcpkg/packages/vcpkg-pkgconfig-get-modules_x64-windows/share/vcpkg-pkgconfig-get-modules/copyright
-- Performing post-build validation
Starting submission of vcpkg-pkgconfig-get-modules:x64-windows@2024-04-03 to 1 binary cache(s) in the background
Elapsed time to handle vcpkg-pkgconfig-get-modules:x64-windows: 128 ms
vcpkg-pkgconfig-get-modules:x64-windows package ABI: a27ea8fb61596d6c641e670c7f9847378f7613e33b49359b24a3280490965665
Installing 139/142 zstd:x64-windows@1.5.7...
Building zstd:x64-windows@1.5.7...
Downloading https://github.com/facebook/zstd/archive/v1.5.7.tar.gz -> facebook-zstd-v1.5.7.tar.gz
Successfully downloaded facebook-zstd-v1.5.7.tar.gz
-- Extracting source D:/Mestrado/Dissertation/labs/vcpkg/downloads/facebook-zstd-v1.5.7.tar.gz
-- Applying patch no-static-suffix.patch
-- Applying patch fix-emscripten-and-clang-cl.patch
-- Applying patch fix-windows-rc-compile.patch
-- Using source at D:/Mestrado/Dissertation/labs/vcpkg/buildtrees/zstd/src/v1.5.7-bb6cae2b2e.clean
-- Found external ninja('1.12.1').
-- Configuring x64-windows
-- Building x64-windows-dbg
-- Building x64-windows-rel
-- Fixing pkgconfig file: D:/Mestrado/Dissertation/labs/vcpkg/packages/zstd_x64-windows/lib/pkgconfig/libzstd.pc
-- Using cached msys2-mingw-w64-x86_64-pkgconf-1~2.4.3-1-any.pkg.tar.zst
-- Using cached msys2-msys2-runtime-3.6.2-2-x86_64.pkg.tar.zst
-- Using msys root at D:/Mestrado/Dissertation/labs/vcpkg/downloads/tools/msys2/9272adbcaf19caef
-- Fixing pkgconfig file: D:/Mestrado/Dissertation/labs/vcpkg/packages/zstd_x64-windows/debug/lib/pkgconfig/libzstd.pc
-- Installing: D:/Mestrado/Dissertation/labs/vcpkg/packages/zstd_x64-windows/share/zstd/usage
-- Performing post-build validation
Starting submission of zstd:x64-windows@1.5.7 to 1 binary cache(s) in the background
Elapsed time to handle zstd:x64-windows: 17 s
zstd:x64-windows package ABI: 9cac722b6a2b9b70af4121ea44efe5984c10e09525e233058abf3a0cd922259d
Completed submission of pkgconf:x64-windows@2.4.3#1 to 1 binary cache(s) in 244 ms
Completed submission of vcpkg-pkgconfig-get-modules:x64-windows@2024-04-03 to 1 binary cache(s) in 56 ms
Installing 140/142 qt5-base[core,openssl]:x64-windows@5.15.16#4...
Building qt5-base[core,openssl]:x64-windows@5.15.16#4...
CMake Warning at scripts/cmake/vcpkg_buildpath_length_warning.cmake:4 (message):
  qt5-base's buildsystem uses very long paths and may fail on your system.

  We recommend moving vcpkg to a short path such as 'C:\src\vcpkg' or using
  the subst command.
Call Stack (most recent call first):
  ports/qt5-base/portfile.cmake:1 (vcpkg_buildpath_length_warning)
  scripts/ports.cmake:206 (include)


-- Figuring out qt target mkspec. Target arch x64
-- Target mkspec set to: win32-msvc
-- Host mkspec not set. Qt's own buildsystem will try to figure out the host system
CMake Warning at scripts/cmake/vcpkg_buildpath_length_warning.cmake:4 (message):
  qt5-base's buildsystem uses very long paths and may fail on your system.

  We recommend moving vcpkg to a short path such as 'C:\src\vcpkg' or using
  the subst command.
Call Stack (most recent call first):
  ports/qt5-base/cmake/qt_download_submodule.cmake:13 (vcpkg_buildpath_length_warning)
  ports/qt5-base/portfile.cmake:57 (qt_download_submodule)
  scripts/ports.cmake:206 (include)


Downloading qtbase-everywhere-opensource-src-5.15.16.tar.xz, trying https://download.qt.io/archive/qt/5.15/5.15.16/submodules/qtbase-everywhere-opensource-src-5.15.16.tar.xz
Successfully downloaded qtbase-everywhere-opensource-src-5.15.16.tar.xz
-- Extracting source D:/Mestrado/Dissertation/labs/vcpkg/downloads/qtbase-everywhere-opensource-src-5.15.16.tar.xz
-- Applying patch patches/0001-CVE-2023-51714-qtbase-5.15.diff
-- Applying patch patches/0002-CVE-2023-51714-qtbase-5.15.diff
-- Applying patch patches/CVE-2024-25580-qtbase-5.15.diff
-- Applying patch patches/CVE-2024-39936-qtbase-5.15.patch
-- Applying patch patches/winmain_pro.patch
-- Applying patch patches/windows_prf.patch
-- Applying patch patches/qt_app.patch
-- Applying patch patches/gui_configure.patch
-- Applying patch patches/xlib.patch
-- Applying patch patches/egl.patch
-- Applying patch patches/qtbug_96392.patch
-- Applying patch patches/mysql_plugin_include.patch
-- Applying patch patches/mysql-configure.patch
-- Applying patch patches/patch-qtbase-memory_resource.diff
-- Applying patch patches/Qt5BasicConfig.patch
-- Applying patch patches/Qt5PluginTarget.patch
-- Applying patch patches/create_cmake.patch
-- Applying patch patches/Qt5GuiConfigExtras.patch
-- Applying patch patches/fix_angle.patch
-- Applying patch patches/mingw9.patch
-- Applying patch patches/qmake-arm64.patch
-- Using source at D:/Mestrado/Dissertation/labs/vcpkg/buildtrees/qt5-base/src/5.15.16-ce7e700ea9.clean
-- Found external ninja('1.12.1').
-- Getting CMake variables for x64-windows
-- Loading CMake variables from D:/Mestrado/Dissertation/labs/vcpkg/buildtrees/qt5-base/cmake-get-vars_C_CXX-x64-windows.cmake.log
-- Using cached msys2-mingw-w64-x86_64-pkgconf-1~2.4.3-1-any.pkg.tar.zst
-- Using cached msys2-msys2-runtime-3.6.2-2-x86_64.pkg.tar.zst
-- Using msys root at D:/Mestrado/Dissertation/labs/vcpkg/downloads/tools/msys2/9272adbcaf19caef
-- Configuring x64-windows-dbg
-- Configuring x64-windows-dbg done
-- Configuring x64-windows-rel
-- Configuring x64-windows-rel done
Downloading https://github.com/lexxmark/winflexbison/releases/download/v2.5.24/win_flex_bison-2.5.24.zip -> win_flex_bison-2.5.24.zip
Successfully downloaded win_flex_bison-2.5.24.zip
-- Building x64-windows-dbg
-- Fixing makefile installation path x64-windows-dbg
-- Installing x64-windows-dbg
-- Package x64-windows-dbg done
-- Building x64-windows-rel
-- Fixing makefile installation path x64-windows-rel
-- Installing x64-windows-rel
-- Package x64-windows-rel done
-- Installing: D:/Mestrado/Dissertation/labs/vcpkg/packages/qt5-base_x64-windows/share/qt5-base/copyright
-- Performing post-build validation
Starting submission of qt5-base[core,openssl]:x64-windows@5.15.16#4 to 1 binary cache(s) in the background
Elapsed time to handle qt5-base:x64-windows: 18 min
qt5-base:x64-windows package ABI: bfad0ce43fe7cac5a51dc6006b7c05e0297db29cf713719e30b005447807fc3d
Completed submission of zstd:x64-windows@1.5.7 to 1 binary cache(s) in 341 ms
Installing 141/142 sqlite3[core,json1]:x64-windows@3.49.2...
Building sqlite3[core,json1]:x64-windows@3.49.2...
Downloading https://sqlite.org/2025/sqlite-autoconf-3490200.tar.gz -> sqlite-autoconf-3490200.zip
Successfully downloaded sqlite-autoconf-3490200.zip
-- Extracting source D:/Mestrado/Dissertation/labs/vcpkg/downloads/sqlite-autoconf-3490200.zip
-- Applying patch fix-arm-uwp.patch
-- Applying patch add-config-include.patch
-- Using source at D:/Mestrado/Dissertation/labs/vcpkg/buildtrees/sqlite3/src/nf-3490200-5690851098.clean
-- Found external ninja('1.12.1').
-- Configuring x64-windows
-- Building x64-windows-dbg
-- Building x64-windows-rel
-- Fixing pkgconfig file: D:/Mestrado/Dissertation/labs/vcpkg/packages/sqlite3_x64-windows/lib/pkgconfig/sqlite3.pc
-- Using cached msys2-mingw-w64-x86_64-pkgconf-1~2.4.3-1-any.pkg.tar.zst
-- Using cached msys2-msys2-runtime-3.6.2-2-x86_64.pkg.tar.zst
-- Using msys root at D:/Mestrado/Dissertation/labs/vcpkg/downloads/tools/msys2/9272adbcaf19caef
-- Fixing pkgconfig file: D:/Mestrado/Dissertation/labs/vcpkg/packages/sqlite3_x64-windows/debug/lib/pkgconfig/sqlite3.pc
-- Installing: D:/Mestrado/Dissertation/labs/vcpkg/packages/sqlite3_x64-windows/share/sqlite3/usage
-- Performing post-build validation
Starting submission of sqlite3[core,json1]:x64-windows@3.49.2 to 1 binary cache(s) in the background
Elapsed time to handle sqlite3:x64-windows: 1.5 min
sqlite3:x64-windows package ABI: 6a07361c0f05ad541f667e32665932e41116632f727c60dbc169260a405e2817
Installing 142/142 colmap[core,cuda,gui,tests]:x64-windows@3.11.1#3...
Building colmap[core,cuda,gui,tests]:x64-windows@3.11.1#3...
Downloading https://github.com/colmap/colmap/archive/3.11.1.tar.gz -> colmap-colmap-3.11.1.tar.gz
Successfully downloaded colmap-colmap-3.11.1.tar.gz
-- Extracting source D:/Mestrado/Dissertation/labs/vcpkg/downloads/colmap-colmap-3.11.1.tar.gz
-- Applying patch no-glu.diff
-- Applying patch fix-flann.patch
-- Using source at D:/Mestrado/Dissertation/labs/vcpkg/buildtrees/colmap/src/3.11.1-c13b079f3d.clean
-- Found external ninja('1.12.1').
-- Configuring x64-windows-dbg
-- Configuring x64-windows-rel
-- Building x64-windows-dbg
CMake Error at scripts/cmake/vcpkg_execute_build_process.cmake:134 (message):
    Command failed: D:/Mestrado/Dissertation/labs/vcpkg/downloads/tools/cmake-3.30.1-windows/cmake-3.30.1-windows-i386/bin/cmake.exe --build . --config Debug --target install -- -v -j13
    Working Directory: D:/Mestrado/Dissertation/labs/vcpkg/buildtrees/colmap/x64-windows-dbg
    See logs for more information:
      D:\Mestrado\Dissertation\labs\vcpkg\buildtrees\colmap\install-x64-windows-dbg-out.log

Call Stack (most recent call first):
  installed/x64-windows/share/vcpkg-cmake/vcpkg_cmake_build.cmake:74 (vcpkg_execute_build_process)
  installed/x64-windows/share/vcpkg-cmake/vcpkg_cmake_install.cmake:16 (vcpkg_cmake_build)
  ports/colmap/portfile.cmake:81 (vcpkg_cmake_install)
  scripts/ports.cmake:206 (include)


error: building colmap:x64-windows failed with: BUILD_FAILED
See https://learn.microsoft.com/vcpkg/troubleshoot/build-failures?WT.mc_id=vcpkg_inproduct_cli for more information.
Elapsed time to handle colmap:x64-windows: 11 min
Please ensure you're using the latest port files with `git pull` and `vcpkg update`.
Then check for known issues at:
  https://github.com/microsoft/vcpkg/issues?q=is%3Aissue+is%3Aopen+in%3Atitle+colmap
You can submit a new issue at:
  https://github.com/microsoft/vcpkg/issues/new?title=[colmap]+Build+error+on+x64-windows&body=Copy%20issue%20body%20from%20D%3A%2FMestrado%2FDissertation%2Flabs%2Fvcpkg%2Finstalled%2Fvcpkg%2Fissue_body.md

Completed submission of qt5-base[core,openssl]:x64-windows@5.15.16#4 to 1 binary cache(s) in 1.6 min
Completed submission of sqlite3[core,json1]:x64-windows@3.49.2 to 1 binary cache(s) in 424 ms
(base) PS D:\Mestrado\Dissertation\labs\vcpkg>